In [1]:
import sys
import os
import numpy as np
from sklearn import metrics
sys.path.append("..")
from SINC_functions import *

In [2]:
# import warnings
# warnings.filterwarnings("ignore",category = DeprecationWarning, module = user_ns.get("__name__"))

def TPR_FPR_B(phi,B):
    B_est = np.absolute(phi) > 0.050
    B_true = B != 0

    TP = np.sum(((phi > 0.50) == 1) & ((B != 0) == 1))  * 1.0
    FP = np.sum(((phi > 0.50) == 1) & ((B != 0) == 0))  * 1.0
    FN = np.sum(((phi > 0.50) == 0) & ((B != 0) == 1))  * 1.0
    TN = np.sum(((phi > 0.50) == 0) & ((B != 0) == 0))  * 1.0
    NN = np.sum(B_true == 0) * 1.0
    NP = np.sum(B_true == 1) * 1.0

    prec = TP / (TP + FP)
    recall = TP / (TP + FN)

    F1 = 2*(prec*recall) / (prec + recall)

    MCC = ((TP*TN) - (FP*FN))/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

    TPR = TP/NP
    FPR = FP/(FP + NN)
    return(TPR,FPR,F1,MCC)

## get shapes for simulation
B_true = np.genfromtxt("./SimulatedData/Band/B1.csv")
X_true = np.genfromtxt("./SimulatedData/Band/x1.csv")

N = X_true.shape[0]
P = X_true.shape[1]
Q = B_true.shape[1]


reps = 10
goal = .10
adj_est_all = np.zeros((P, P, reps))
EZ_all = np.zeros((P, P, reps))
B_est_all = np.zeros((P,Q,reps))
phi_all = np.copy(B_est_all)
spars = np.zeros(reps)
v0s = np.linspace(.0001, .01, reps)
TPR = np.zeros(reps)
FPR = np.zeros(reps)
from itertools import chain

results = np.zeros((25, 5))
results_B = np.zeros((25,4))
total_iters = np.zeros(reps)

Z_track_all = []
omega_track_all = []
B_track_all = []

for sim in range(25):
    TPR = np.zeros(reps)
    FPR = np.zeros(reps)
    iters = np.zeros(reps)
    spars = np.zeros(reps)
    
    v1 = 10
    lamb = 150
    vB = 1
    max_iters = 3
    tol_prec = 0.01
    tol_B = 0.01
    cpus = 1

    print("**********************start iteration " + str(sim) + "*************************")
    x = np.genfromtxt("./SimulatedData/Band/x" + str(1 + sim) + ".csv")
    m = np.genfromtxt("./SimulatedData/Band/m" + str(1 + sim) + ".csv")
    adj_true = np.genfromtxt("./SimulatedData/Band/adj" + str(1 + sim) + ".csv")
    B_true = np.genfromtxt("./SimulatedData/Band/B" + str(1 + sim) + ".csv")

    for v in range(reps):
        
        v0 = v0s[v]

        omega, EZ, phi,B,iters_total  = SINC(x, m, v0, v1, lamb, vB, max_iters, tol_prec, tol_B, cpus)
        
        B_est_all[:,:,v] = B
        phi_all[:,:,v] = phi
        EZ_all[:, :, v] = EZ
        adj_est = (EZ > 0.50)*1.0
        spars[v] = np.sum(adj_est) / 2
        TPR[v], FPR[v] = TPR_FPR(adj_true, adj_est)
        if np.sum(adj_est) / 2 <= 5:
            break

    adj_est_all = (EZ_all > 0.50) * 1
    ind = np.argmin(np.absolute((np.sum(adj_est_all, (0,1)) / (12 * 11 / 2.0) / 2) - .10))
    EZ = EZ_all[:, :, ind]
    phi_ind = phi_all[:,:,ind]
    total_iters[v] = iters[ind]


    TPR = np.append(1, np.append(TPR, 0))
    FPR = np.append(1, np.append(FPR, 0))
    auc = metrics.auc(np.array(FPR),np.array(TPR))

    auc_v0 = 0
    n = len(TPR)
    for n in range(n - 1):
        b1 = TPR[n]
        b2 = TPR[n + 1]
        h = FPR[n] - FPR[n + 1]
        auc_v0 += (b1 + b2) * h / 2

    adj_est = adj_est_all[:, :, ind]
    TPR, FPR = TPR_FPR(adj_true, adj_est)
    f1, mcc = F1(adj_true, adj_est)
    results[sim,] = [TPR, FPR, f1, mcc, auc]
    B_edges_true = (B_true != 0)*1.0
    B_edges_est = (phi_ind > 0.50) *1
    results_B[sim,] = Performance_B(B_edges_true,B_edges_est)

    np.savetxt("./Results/NetworkResults_Band.csv",results, delimiter=",")
    np.savetxt("./Results/CoefficientResults_Band.csv",results_B, delimiter=",")


**********************start iteration 0*************************
('Finished Iteration 1: ', 'change in Omega,B,Z', 0.462880369831428, 0.7378843589487303, 3.1348969187984697)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.5551923406941089, 0.17139155831016228, 1.2905798200489562)
('Finished Iteration 3: ', 'change in Omega,B,Z', 0.18674804930333333, 0.170613999672722, 0.757250371881425)
('v0 = ', 0.0001, 'Sparsity = ', 0.4513888888888889)
('Finished Iteration 1: ', 'change in Omega,B,Z', 0.47647336478058455, 0.7378843589487303, 3.136725850838088)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.5414092224972428, 0.1718844783762482, 1.3331450155989535)
('Finished Iteration 3: ', 'change in Omega,B,Z', 0.17315057379909693, 0.16906257579236456, 0.7715950428536464)
('v0 = ', 0.0012000000000000001, 'Sparsity = ', 0.3125)
('Finished Iteration 1: ', 'change in Omega,B,Z', 0.47684418959265107, 0.7378843589487303, 3.1374822010294343)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.52